In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("pokemon_gen_1_to_8.csv")
pok = pd.DataFrame(data)
pok.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Attack,Sp. Defense,Speed,Total,Average,Generation
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,318,53.00,1
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,405,67.50,1
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,525,87.50,1
3,3,Venusaur (Mega Venusaur),Grass,Poison,80,100,123,122,120,80,625,104.17,1
4,4,Charmander,Fire,NaN,39,52,43,60,50,65,309,51.50,1


In [4]:
ou = pd.read_csv("gen8_ou.csv")
ou.columns = ["Name", "Usage Rate"]
tier = "OU"
ou["Tier"] = tier
#for col in ou.columns:
 #   print(col)
ou.head()

,Name,Usage Rate,Tier
0,Clefable,56.53422%,OU
1,Corviknight,31.22480%,OU
2,Toxapex,28.25718%,OU
3,Hippowdon,25.68515%,OU
4,Ferrothorn,25.17680%,OU


In [5]:
uu = pd.read_csv("gen8_uu.csv")
uu.columns = ["Name", "Usage Rate"]
tier = "UU"
uu["Tier"] = tier
uu.head()

,Name,Usage Rate,Tier
0,Noivern,38.70327%,UU
1,Haxorus,31.67690%,UU
2,Sylveon,30.85321%,UU
3,Incineroar,29.24782%,UU
4,Cobalion,26.66785%,UU


In [6]:
ru = pd.read_csv("gen8_ru.csv")
ru.columns = ["Name", "Usage Rate"]
tier = "RU"
ru["Tier"] = tier
ru.head()

,Name,Usage Rate,Tier
0,Steelix,32.52857%,RU
1,Xatu,27.03527%,RU
2,Snorlax,25.02356%,RU
3,Vileplume,24.01253%,RU
4,Passimian,23.81958%,RU


In [7]:
nu = pd.read_csv("gen8_nu.csv")
nu.columns = ["Name", "Usage Rate"]
tier = "NU"
nu["Tier"] = tier
nu.head()

,Name,Usage Rate,Tier
0,Togedemaru,27.07538%,NU
1,Sandaconda,26.42466%,NU
2,Clefairy,25.01305%,NU
3,Cofagrigus,21.84802%,NU
4,Rotom-Fan,20.33243%,NU


In [8]:
pu = pd.read_csv("gen8_pu.csv")
pu.columns = ["Name", "Usage Rate"]
tier = "PU"
pu["Tier"] = tier
pu.head()

,Name,Usage Rate,Tier
0,Throh,37.73567%,PU
1,Mawile,30.25689%,PU
2,Manectric,21.35988%,PU
3,Pawniard,19.70286%,PU
4,Carkol,18.47475%,PU


In [18]:
new_df = pd.concat([ou, uu, ru, nu, pu], axis=0)
#print(len(new_df))
new_df[~new_df.Name.str.contains("Type: Null")]
#print(len(new_df))
#for f in new_df['Name'].unique():
 #   print(new_df[new_df['Name'] == f])


,Name,Usage Rate,Tier
0,Clefable,56.53422%,OU
1,Corviknight,31.22480%,OU
2,Toxapex,28.25718%,OU
3,Hippowdon,25.68515%,OU
4,Ferrothorn,25.17680%,OU
...,...,...,...
202,Wailmer,0.00001%,PU
203,Pidove,0.00001%,PU
204,Squirtle,0.00001%,PU
205,Wooloo,0.00001%,PU
